In [ ]:
import java.sql._
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql.streaming.Trigger
import com.typesafe.config._


In [ ]:
val config = ConfigFactory.load("application.conf").getConfig("com.ram.batch")
val sparkConfig = config.getConfig("spark")
val mysqlConfig = config.getConfig("mysql")
val appName = sparkConfig.getString(BatchConstants.GET_APP_NAME)
println(appName) // prints my-app


In [ ]:
// configure Spark Session
val spark = SparkSession
.builder()
.appName("Twitter Streaming")
.master("local[*]")
.getOrCreate()


In [ ]:
// Define data source
// here, it's the Kafka broker: tweetStream.get_twitter_data()
val data_stream = spark
.readStream // constantly expanding dataframe
.format("kafka")
.option("kafka.bootstrap.servers", "localhost:9092")
.option("subscribe", "data-tweets")
.option("startingOffsets", "latest") // or earliest
.load()

// schema of incoming data:
// schema = StructType(
    // StructField(key, BinaryType, true),
    // StructField(value, BinaryType, true),
    // StructField(topic, StringType, true),
    // StructField(partition, IntegerType, true),
    // StructField(offset, LongType, true),
    // StructField(timestamp, TimestampType, true),
    // StructField(timestampType, IntegerType, true)
    // )


In [ ]:
// watermarking to mark late-arriving data
var data_stream_transformed = data_stream
.withWatermark("timestamp", "1 day")


In [ ]:
// incoming tweet schema
val schema = StructType(Seq(
    StructField("created_at", StringType, true),
    StructField("id", StringType, true),
    StructField("id_str", StringType, true),
    StructField("text", StringType, true),
    StructField("retweet_count", StringType, true),
    StructField("favorite_count", StringType, true),
    StructField("favorited", StringType, true),
    StructField("retweeted", StringType, true),
    StructField("lang", StringType, true),
    StructField("location", StringType, true)
))


In [ ]:
// data transformations
data_stream_transformed = data_stream_transformed
.selectExpr("CAST(value AS STRING) as json") // byte - -> string
.select(from_json(col("json"), schema=schema).as("tweet")) // json string - -> table defined by schema
  .selectExpr( // data type casts
              "tweet.created_at",
               "cast (tweet.id as long)",
               "tweet.id_str",
               "tweet.text",
               "cast (tweet.retweet_count as integer)",
               "cast (tweet.favorite_count as integer)",
               "cast(tweet.favorited as boolean)",
               "cast(tweet.retweeted as boolean)",
               "tweet.lang as language_code"
              )
// result = json tweets formatted in df


In [ ]:
// group tweets by language code & count number of likes per tweet author
data_stream_transformed = data_stream_transformed
.groupBy("language_code")
.agg(sum("favorite_count"), count("id"))


In [ ]:
// define database urls
val url = "jdbc:postgresql://192.168.10.157:5432/pipeline"
val user = "pipeline"
val pw = "Ch4ng3Me!"
val jdbcWriter = new PostgreSqlSink(url, user, pw)


In [ ]:
// create trigger for stream updates
val writeData = data_stream_transformed
.writeStream
.option("checkpointLocation", "/Users/dbg/Desktop/checkpoint") // checkpoint prevents loss of state during downtime
.foreach(writer)
.trigger(Trigger.ProcessingTime("10 seconds"))
.outputMode("update")
.start()


In [ ]:
// define postgresql sink


class PostgreSqlSink(url: String, user: String, pwd: String) extends org.apache.spark.sql.ForeachWriter[org.apache.spark.sql.Row] {
    val driver = "org.postgresql.Driver"
    var connection: java.sql.Connection = _
    var statement: java.sql.PreparedStatement = _

    def open(partitionId: Long, version: Long): Boolean = {
        Class.forName(driver)
        connection = java.sql.DriverManager.getConnection(url, user, pwd)
        connection.setAutoCommit(false)
        statement = connection.prepareStatement(v_sql)
        true
    }

    // process each tweet(row)
    // copied directly from: https: // sonra.io/spark/advanced-spark-structured-streaming-aggregations-joins-checkpointing /
    def process(value: org.apache.spark.sql.Row): Unit = {
        statement.executeUpdate(
            "DELETE FROM stream_tweets WHERE stream_tweets.language_code = '" +
            value(0) + "' ; "
        )
        statement.executeUpdate(
            "INSERT INTO stream_tweets VALUES('" + value(0) + "', " + value(
                1) + ", " + value(2) + ", '" + value(3) + "');"
        )
    }
    def close(errorOrNull: Throwable): Unit = {
        connection.commit()
        connection.close
    }
}


In [ ]:
print("Starting stream...")
writeData.awaitTermination(20) // trigger executes ever 20 seconds


In [ ]:
// get status of stream
println(writeData.status)
println(writeData.exception)
println(writeData.lastProgress)
writeData.explain()
